# Iris Classification Model

Laporan akhir ini merangkum seluruh tahapan pembuatan model klasifikasi bunga Iris, mulai dari persiapan data, pelatihan model, visualisasi hasil, hingga prediksi data baru. Notebook ini dapat dijalankan secara berurutan.

In [2]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'pandas'

## 1. Persiapan Data
Memuat dataset Iris bawaan scikit-learn, membagi data menjadi latih dan uji, serta menyimpan ke file CSV.

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
df = pd.concat([iris.data, iris.target.rename('target')], axis=1)

# Simpan data
df.to_csv('iris_full.csv', index=False)

# Split
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['target'])
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

print("Data disiapkan: train.csv, test.csv")

## 2. Pelatihan Model
Membaca `train.csv`, melakukan scaling, dan melatih model MLP selama 100 epoch.

In [ ]:
# Definisi model
class IrisNet(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=16, output_dim=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x):
        return self.net(x)

# Load and preprocess
df_train = pd.read_csv('train.csv')
X = df_train.drop(columns='target').values
y = df_train['target'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler().fit(X_train)
X_train, X_val = scaler.transform(X_train), scaler.transform(X_val)

# Simpan scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# DataLoader
train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_ds   = TensorDataset(torch.tensor(X_val, dtype=torch.float32),   torch.tensor(y_val, dtype=torch.long))
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=16)

# Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model      = IrisNet().to(device)
criterion  = nn.CrossEntropyLoss()
optimizer  = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
n_epochs = 100
train_losses, val_accs = [], []
best_val_acc = 0.0

for epoch in range(1, n_epochs+1):
    # Train
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    train_loss = total_loss / len(train_loader.dataset)

    # Validate
    model.eval()
    correct = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb).argmax(dim=1)
            correct += (preds == yb).sum().item()
    val_acc = correct / len(val_loader.dataset)

    train_losses.append(train_loss)
    val_accs.append(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{n_epochs} - Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}")

# Save final model
torch.save(model.state_dict(), 'final_model.pth')
print(f"Training selesai. Best Val Acc = {best_val_acc:.4f}")

## 3. Visualisasi Hasil

In [ ]:
# Plot Loss dan Akurasi
epochs = range(1, n_epochs+1)

plt.figure()
plt.plot(epochs, train_losses)
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

plt.figure()
plt.plot(epochs, val_accs)
plt.title('Validation Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

## 4. Prediksi Data Baru

In [ ]:
# Contoh data baru
new_samples = np.array([[5.1, 3.5, 1.4, 0.2],
                        [6.7, 3.0, 5.2, 2.3]])

# Load scaler & model
scaler = pickle.load(open('scaler.pkl', 'rb'))
model = IrisNet().to(device)
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.eval()

# Preprocess & predict
X_new = scaler.transform(new_samples)
with torch.no_grad():
    preds = model(torch.tensor(X_new, dtype=torch.float32).to(device)).argmax(dim=1).cpu().numpy()

# Tampilkan hasil
for sample, p in zip(new_samples, preds):
    print(f"Features: {sample} -> Predicted class: {iris.target_names[p]}")